In [1]:
from pathlib import Path
import sys
import os
import geopandas as gpd
import pandas as pd

scripts_dir = Path("..").joinpath("src")
if scripts_dir not in sys.path:
    sys.path.insert(0, scripts_dir.resolve().as_posix())
from helpers.mapshaper import Mapshaper
from helpers.tippcanoe import mbtileGeneration
from helpers.mapbox_uploader import uploadToMapbox
from data_commons.loader import load_regions

In [2]:
source_dir = Path("..").joinpath("data", "mpa_intermediate").resolve()
file = source_dir.joinpath("./merged_mpa_all.json")
tileset = source_dir.joinpath("./merged_mpa_all.mbtiles")

In [18]:
Mapshaper(16).input(
    [source_dir.joinpath("mpa_intermediate/mpa_intermediate.shp").as_posix()]
).filter_fields(fields="WDPAID,NAME,PA_DEF,REP_M_AREA,PARENT_ISO").clean(
    allow_overlaps=True, rewind=True
).simplify(
    "dp 10% keep-shapes planar"
).clean(
    allow_overlaps=True
).output(
    source_dir.joinpath("mpa_intermediate_simp.json").as_posix(), force=True, format="geojson"
).execute()

Allocating 16 GB of heap memory
[clean] Retained 18,732 of 18,732 features
[simplify] Repaired 135,759 intersections; 167,976 intersections could not be repaired
[clean] Retained 18,732 of 18,732 features
[o] Wrote /home/mambauser/data/mpa_intermediate/mpa_intermediate_simp.json


CompletedProcess(args='mapshaper-xl 16gb -i /home/mambauser/data/mpa_intermediate/mpa_intermediate/mpa_intermediate.shp  -filter-fields fields=WDPAID,NAME,PA_DEF,REP_M_AREA,PARENT_ISO -clean allow-overlaps rewind -simplify dp 10% keep-shapes planar -clean allow-overlaps -o /home/mambauser/data/mpa_intermediate/mpa_intermediate_simp.json force format=geojson', returncode=0)

In [21]:
mbtileGeneration(source_dir.joinpath("mpa_intermediate_simp.json"))

For layer 0, using name "mpa_intermediate_simp"
/home/mambauser/data/mpa_intermediate/mpa_intermediate_simp.json:2037: Found ] at top level
/home/mambauser/data/mpa_intermediate/mpa_intermediate_simp.json:449: Reached EOF without all containers being closed
In JSON object {"type":"FeatureCollection","features":[]}
18732 features, 65660163 bytes of geometry, 325522 bytes of separate metadata, 675303 bytes of string pool
Choosing a maxzoom of -z3 for features about 33084 feet (10084 meters) apart
Choosing a maxzoom of -z10 for resolution of about 345 feet (105 meters) within features
  99.9%  10/42/292   


PosixPath('/home/mambauser/data/mpa_intermediate/mpa_intermediate_simp.mbtiles')

In [22]:
uploadToMapbox(
    source_dir.joinpath("mpa_intermediate_simp.mbtiles"),
    "mpas_wdpa",
    os.environ["MAPBOX_USER"],
    os.environ["MAPBOX_TOKEN"],
)

upload: ../data/mpa_intermediate/mpa_intermediate_simp.mbtiles to s3://tilestream-tilesets-production/5e/_pending/cvat4dd2vqm1aq41tzh4l0slc/skytruth


Linking tileset to Mapbox: 100%|██████████| 100/100 [03:10<00:00,  1.91s/it]


True

In [11]:
regions_df = pd.DataFrame(
    [
        {"region_id": data["region_iso"], "location_id": iso}
        for data in load_regions().get("data", [])
        for iso in data["country_iso_3s"]
    ]
)

In [18]:
eez_dir = Path("..").joinpath("data", "eez_intermediate", "eez_intermediate").resolve()
eez_data = gpd.read_file(eez_dir.joinpath("eez_intermediate.shp").as_posix())

In [14]:
Mapshaper(8).input([eez_dir.joinpath("eez_intermediate.shp").as_posix()]).filter_fields(
    fields=','.join(["GEONAME", "POL_TYPE", "ISO_SOV1", "ISO_SOV2", "ISO_SOV3"])
).output(
    eez_dir.joinpath("eez_intermediate.json").as_posix(), force=True, format="geojson"
).execute()

Allocating 8 GB of heap memory
[o] Wrote /home/mambauser/data/eez_intermediate/eez_intermediate/eez_intermediate.json


CompletedProcess(args='mapshaper-xl 8gb -i /home/mambauser/data/eez_intermediate/eez_intermediate/eez_intermediate.shp  -filter-fields fields=GEONAME,POL_TYPE,ISO_SOV1,ISO_SOV2,ISO_SOV3 -o /home/mambauser/data/eez_intermediate/eez_intermediate/eez_intermediate.json force format=geojson', returncode=0)

In [15]:
gpd.GeoDataFrame(
    pd.merge(
        eez_data,
        regions_df,
        how="left",
        left_on="ISO_SOV1",
        right_on="location_id",
        sort=True,
        copy=True,
    ),
    crs=eez_data.crs,
).to_file(filename=eez_dir.joinpath("eez_intermediate_regions.shp").as_posix(), driver="ESRI Shapefile")

/tmp/ipykernel_1179/274243943.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gpd.GeoDataFrame(


In [19]:
Mapshaper(16).input([eez_dir.joinpath("eez_intermediate_regions.shp").as_posix()]).dissolve2(
    fields="region_id"
).output(eez_dir.joinpath("regions.json").as_posix(), force=True, format="geojson").execute()

Allocating 16 GB of heap memory
[dissolve2] Dissolved 282 features into 8 features
[o] Wrote /home/mambauser/data/eez_intermediate/eez_intermediate/regions.json


CompletedProcess(args='mapshaper-xl 16gb -i /home/mambauser/data/eez_intermediate/eez_intermediate/eez_intermediate_regions.shp  -dissolve2 fields=region_id -o /home/mambauser/data/eez_intermediate/eez_intermediate/regions.json force format=geojson', returncode=0)

In [20]:
mbtileGeneration(eez_dir.joinpath("eez_v11.json"))

For layer 0, using name "eez_v11"
/home/mambauser/data/eez_intermediate/eez_intermediate/eez_v11.json:3: Found ] at top level
/home/mambauser/data/eez_intermediate/eez_intermediate/eez_v11.json:23: Reached EOF without all containers being closed
In JSON object {"type":"FeatureCollection","features":[]}
282 features, 62937025 bytes of geometry, 5323 bytes of separate metadata, 13435 bytes of string pool
Choosing a maxzoom of -z0 for features about 2511227 feet (765422 meters) apart
Choosing a maxzoom of -z9 for resolution of about 825 feet (251 meters) within features
  99.9%  9/201/377  


PosixPath('/home/mambauser/data/eez_intermediate/eez_intermediate/eez_v11.mbtiles')

In [21]:
uploadToMapbox(
    eez_dir.joinpath("eez_v11.mbtiles"),
    "eez_v11",
    os.environ["MAPBOX_USER"],
    os.environ["MAPBOX_TOKEN"],
)

upload: ../data/eez_intermediate/eez_intermediate/eez_v11.mbtiles to s3://tilestream-tilesets-production/dd/_pending/uvt9z3qcvqp10i417h45j0slc/skytruth


Linking tileset to Mapbox: 100%|██████████| 100/100 [03:10<00:00,  1.91s/it]


True

In [22]:
mbtileGeneration(eez_dir.joinpath("regions.json"))

For layer 0, using name "regions"
/home/mambauser/data/eez_intermediate/eez_intermediate/regions.json:2: Found ] at top level
/home/mambauser/data/eez_intermediate/eez_intermediate/regions.json:3: Reached EOF without all containers being closed
In JSON object {"type":"FeatureCollection","features":[]}
8 features, 60659758 bytes of geometry, 40 bytes of separate metadata, 85 bytes of string pool
Choosing a maxzoom of -z0 for features about 26898822 feet (8198761 meters) apart
Choosing a maxzoom of -z9 for resolution of about 901 feet (274 meters) within features
  99.9%  9/219/232  


PosixPath('/home/mambauser/data/eez_intermediate/eez_intermediate/regions.mbtiles')

In [23]:
uploadToMapbox(
    eez_dir.joinpath("regions.mbtiles"),
    "regions",
    os.environ["MAPBOX_USER"],
    os.environ["MAPBOX_TOKEN"],
)

upload: ../data/eez_intermediate/eez_intermediate/regions.mbtiles to s3://tilestream-tilesets-production/bb/_pending/4k2k6ua5apt1l490gzjbj0slc/skytruth


Linking tileset to Mapbox: 100%|██████████| 100/100 [03:10<00:00,  1.91s/it]


True

In [3]:
mpa_atlas = source_dir.parent.joinpath("mpaatlas_intermediate", "mpaatlas_intermediate")
mpa_atlas

PosixPath('/home/mambauser/data/mpaatlas_intermediate/mpaatlas_intermediate')

In [4]:
mbtileGeneration(mpa_atlas.joinpath("mpaatlas_intermediate.shp"))

Allocating 50 GB of heap memory
[clean] Retained 808 of 810 features
[o] Wrote /home/mambauser/data/mpaatlas_intermediate/mpaatlas_intermediate/mpaatlas_intermediate.json
For layer 0, using name "mpaatlas_intermediate"
/home/mambauser/data/mpaatlas_intermediate/mpaatlas_intermediate/mpaatlas_intermediate.json:293: Reached EOF without all containers being closed
In JSON object {"type":"FeatureCollection","features":[]}
/home/mambauser/data/mpaatlas_intermediate/mpaatlas_intermediate/mpaatlas_intermediate.json:20: Found ] at top level
808 features, 33450840 bytes of geometry, 24231 bytes of separate metadata, 43441 bytes of string pool
Choosing a maxzoom of -z0 for features about 314630 feet (95899 meters) apart
Choosing a maxzoom of -z12 for resolution of about 98 feet (30 meters) within features
  99.9%  12/4079/2746  


PosixPath('/home/mambauser/data/mpaatlas_intermediate/mpaatlas_intermediate/mpaatlas_intermediate.mbtiles')

In [5]:
uploadToMapbox(
    mpa_atlas.joinpath("mpaatlas_intermediate.mbtiles"),
    "mpa_atlas",
    os.environ["MAPBOX_USER"],
    os.environ["MAPBOX_TOKEN"],
)

upload: ../data/mpaatlas_intermediate/mpaatlas_intermediate/mpaatlas_intermediate.mbtiles to s3://tilestream-tilesets-production/e5/_pending/uzkzdbzgbpn14gk02ior8zvlc/skytruth


Linking tileset to Mapbox: 100%|██████████| 100/100 [05:41<00:00,  3.41s/it]


True

In [10]:
protectedSeas = Path("..").joinpath("data", "protectedseas_intermediate", "protectedseas_intermediate").resolve()

In [14]:
Mapshaper(16).input([protectedSeas.joinpath("protectedseas_intermediate.shp").as_posix()]).clean(
    allow_overlaps=True, rewind=True
).simplify("dp 10% keep-shapes planar").clean(allow_overlaps=True).output(
    protectedSeas.joinpath("protectedseas.json").as_posix(), force=True, format="geojson"
).execute()

Allocating 16 GB of heap memory
[clean] Retained 10,879 of 10,879 features
[simplify] Repaired 110,498 intersections; 22,095 intersections could not be repaired
[clean] Retained 10,879 of 10,879 features
[o] Wrote /home/mambauser/data/protectedseas_intermediate/protectedseas_intermediate/protectedseas.json


CompletedProcess(args='mapshaper-xl 16gb -i /home/mambauser/data/protectedseas_intermediate/protectedseas_intermediate/protectedseas_intermediate.shp  -clean allow-overlaps rewind -simplify dp 10% keep-shapes planar -clean allow-overlaps -o /home/mambauser/data/protectedseas_intermediate/protectedseas_intermediate/protectedseas.json force format=geojson', returncode=0)

In [15]:
mbtileGeneration(protectedSeas.joinpath("protectedseas.json"))

For layer 0, using name "protectedseas"
/home/mambauser/data/protectedseas_intermediate/protectedseas_intermediate/protectedseas.json:967: Reached EOF without all containers being closed
In JSON object {"type":"FeatureCollection","features":[]}
/home/mambauser/data/protectedseas_intermediate/protectedseas_intermediate/protectedseas.json:378: Found ] at top level
10879 features, 31017199 bytes of geometry, 377988 bytes of separate metadata, 766466 bytes of string pool
Choosing a maxzoom of -z3 for features about 58402 feet (17801 meters) apart
Choosing a maxzoom of -z11 for resolution of about 245 feet (74 meters) within features
  99.9%  11/831/344    


PosixPath('/home/mambauser/data/protectedseas_intermediate/protectedseas_intermediate/protectedseas.mbtiles')

In [16]:
uploadToMapbox(
    protectedSeas.joinpath("protectedseas.mbtiles"),
    "protected_seas",
    os.environ["MAPBOX_USER"],
    os.environ["MAPBOX_TOKEN"],
)

upload: ../data/protectedseas_intermediate/protectedseas_intermediate/protectedseas.mbtiles to s3://tilestream-tilesets-production/19/_pending/fzxiizy7nqm1l830ze3ck0slc/skytruth


Linking tileset to Mapbox: 100%|██████████| 100/100 [04:12<00:00,  2.53s/it]


True

In [2]:
warmWaterCorals = (
    Path("..")
    .joinpath("data", "habitats_intermediate", "layer", "warm_water_corals.mbtiles")
    .resolve()
)

In [3]:
uploadToMapbox(
    warmWaterCorals,
    "warm_water_corals",
    os.environ["MAPBOX_USER"],
    os.environ["MAPBOX_TOKEN"],
)

upload: ../data/habitats_intermediate/layer/warm_water_corals.mbtiles to s3://tilestream-tilesets-production/36/_pending/2mem3w8f6ku161y4r60lfetlc/skytruth


Linking tileset to Mapbox: 100%|██████████| 100/100 [04:44<00:00,  2.84s/it]


True